# 大语言模型Transformer库-Trainer组件实践

## 基于Trainer优化预训练代码

### 步骤一：导入相关包

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

### 步骤二：加载数据集

In [ ]:
dataset = load_dataset("csv", data_files="/root/pretrains/ChnSentiCorp_htl_all/ChnSentiCorp_htl_all.csv", split="train")
# 使用filter方法对数据集进行过滤，以移除那些"review"字段缺失的样本。
# lambda x: x["review"] is not None是一个匿名函数，它对于数据集中的每个样本x进行评估，
# 如果样本中的"review"字段不是None（即不是缺失或空的），那么这个样本就会通过过滤。
# 过滤后，dataset对象将只包含"review"字段有有效值的样本。
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

### 步骤三：划分数据集

In [ ]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

### 步骤四：数据集预处理

In [ ]:
import torch

tokenizer = AutoTokenizer.from_pretrained("/root/transformers/classmodel/rbt3")
# 定义一个处理函数，该函数将被应用到数据集中的每个样本上。
# 这个函数接收一个examples对象，其中包含了数据集中的一部分样本。
def process_function(examples):
    # 使用tokenizer对"review"字段进行编码，生成模型可理解的格式。
    # max_length=128指定了输出序列的最大长度，超过这个长度的文本将被截断。
    # truncation=True指示Tokenizer进行截断操作。
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples
# 使用map方法将process_function应用到datasets对象中的每个样本上。
# map方法会自动对数据集中的所有样本运行process_function函数。
# batched=True指示map操作应该批处理样本，这通常比逐个处理样本更高效。
# remove_columns=datasets["train"].column_names指定在映射过程中移除原始数据集中的列，只保留处理后的tokenized数据。
tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

### 步骤五：创建模型

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("/root/pretrains/rbt3")

### 步骤六：创建评估函数

In [ ]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metirc = evaluate.load("f1")

def eval_metric(eval_predict):
    #eval_predict拆分成两部分，第一部分是预测值，赋值给predictions，第二部分是实际标签，赋值给labels
    predictions, labels = eval_predict
    # 获取到预测值
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    #是更新准确率指标，把F1分数的计算结果加入到准确率指标的计算中
    acc.update(f1)
    return acc

### 步骤七：创建TrainingArguments

In [ ]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

### 步骤八：创建Trainer

In [ ]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

### 步骤九：模型训练

In [ ]:
trainer.train()

### 步骤十：模型评估

In [ ]:
trainer.evaluate(tokenized_datasets["test"])

### 步骤十一：模型预测

In [ ]:
trainer.predict(tokenized_datasets["test"])

## 完整代码

In [ ]:
# 导入所需的库和模块
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from transformers import DataCollatorWithPadding
from datasets import load_dataset
import torch
import evaluate

# 加载数据集
dataset = load_dataset("csv", data_files="/root/transformers/classdata/ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
datasets = dataset.train_test_split(test_size=0.1)

# 加载预训练模型的分词器
tokenizer = AutoTokenizer.from_pretrained("/root/transformers/classmodel/rbt3")

# 定义处理函数，将文本转换为模型可接受的格式
def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

# 对数据集进行预处理，并返回处理后的结果
tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)

# 加载预训练模型
model = AutoModelForSequenceClassification.from_pretrained("/root/transformers/classmodel/rbt3")

# 加载评估指标
acc_metric = evaluate.load("accuracy")
f1_metirc = evaluate.load("f1")

# 定义评估函数，计算准确率和F1分数
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

# 设置训练参数
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型

# 创建Trainer对象，用于训练和评估模型
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

# 开始训练模型
trainer.train()

# 在测试集上评估模型性能
trainer.evaluate(tokenized_datasets["test"])

# 使用训练好的模型进行预测
trainer.predict(tokenized_datasets["test"])